In [8]:
import json
import time
import re
from api_request.gpt35_turbo_completion import gpt35_turbo_completion


ontology = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
you don't have to find other changed domain-slots if they are not in the list.

hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book stay: 1, 2, 3, etc.
hotel-book day: monday, tuesday, etc.
hotel-book people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book people: 1, 2, 3, etc.
train-depart_time: 20:24, 12:06, etc.
train-arrive_by_time: 05:51, 20:52, etc.

attraction-name: abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area: dontcare, centre, east, north, south, west
attraction-type: architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre

restaurant-name: pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food_type: italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange: dontcare, cheap, moderate, expensive
restaurant-area: centre, east, north, south, west
restaurant-book time: 13:30, 17:11, etc.
restaurant-book day: wednesday, friday, etc.
restaurant-book people: 1, 2, 3, etc.

taxi-destination: copper kettle, magdalene college, lovell lodge
taxi-departure: royal spice, university arms hotel, da vinci pizzeria
taxi-depart_time: 14:45, 11:15, etc.
taxi-arrive_by_time: 15:30, 12:45, etc.
"""

def parse_slot_with_bracket_matching(pred):

    # find all values where they are in the brackets

    # fix for no states
    if pred == "":
        return {}

    pred_slots = []

    # slot_value = pred.split(",")
    slots = re.findall(r'\((.*?)\)', pred)

    for i in slots:
        i = i.replace("(","").replace(")","")
        # for multiple slots in one bracket
        pred_slots.append(i)
    return pred_slots

In [17]:
running_log_path = "expts/230518_1758-gpt35_turbo_5p_v2_retrieve_with_element2aug_0to50/running_log.json"
with open(running_log_path,'r') as f:
    content = json.load(f)
    
# for idx, i in enumerate(content[:200]):
#     if i["pred_status"] == "wrong" and i["predicted_slot_values"] != i["turn_slot_values"]:
#         last_slot_values = {s: v.split('|')[0] for s, v in i['last_slot_values'].items()}
#         if not all(item in last_slot_values.items() for item in i["predicted_slot_values"].items()):
#             print("********** " + str(idx) + " **********")
#             print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
#             print("user uttr  ==== " + i["dialog"]["usr"][-1])
#             print()
#             print("prev_label ==== " + str(i["last_slot_values"]))
#             print()
#             print(i["prompt"].replace(ontology,""))
#     #             print(i["prompt"])
#             print()
#             print("completion ==== " + i["completion"])
#             print()
#             print("prediction ==== " + str(i["predicted_slot_values"]))
#             print("labeling.. ==== " + str(i["turn_slot_values"]))
#             print("\n\n\n\n\n\n\n\n\n")

for idx, i in enumerate(content):
    if i["ag_used"]:
        print("********** " + str(idx) + " **********")
        print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
        print("user uttr  ==== " + i["dialog"]["usr"][-1])
        print()
        print("prev_label ==== " + str(i["last_slot_values"]))
        print()
        for aug in i['ag_used_list']:
            print(f"used ag utt ==== {aug['dialog']['usr'][-1]}")
            print(f"ag_slot ==== {aug['ag_slot']}")
            print(f"turn_slot_values ==== {aug['turn_slot_values']}")
            print()
        print()
        print(i["prompt"].replace(ontology,""))
        print()
        print("completion ==== " + i["completion"])
        print()
        print("prediction ==== " + str(i["predicted_slot_values"]))
        print("labeling.. ==== " + str(i["turn_slot_values"]))
        print("\n\n\n\n\n\n\n\n\n")

********** 4 **********
sys  uttr  ==== 
user uttr  ==== please find a restaurant called nusha .

prev_label ==== {}

used ag utt ==== could you give me the address and postcode of a restaurant called tandoori palace with any price range ?
ag_slot ==== restaurant-pricerange
turn_slot_values ==== {'restaurant-name': 'tandoori', 'restaurant-pricerange': 'dontcare'}



Example #1
[context] 
[system] 
[user] hi ! i am looking for information on the rice house restaurant .
Q: Based on current dialogue states ([context]), system utterance ([system]), and user utterance ([user]), what domain-slots have been changed and what are their values?
A: (restaurant-name = rice house)


Example #2
[context] (restaurant-pricerange = expensive), (restaurant-area = west), (restaurant-food_type = indian)
[system] there are 5 that match you criteria . tandoori palace is a great place , shall i book a table for you ?
[user] could you give me the address and postcode of a restaurant called tandoori palace wit